## Models and validation

#### Packages

In [60]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.exceptions import DataConversionWarning; import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.datasets import fetch_california_housing

#### Splitting data

#### Modelvalidation

In [2]:
cal_house = fetch_california_housing()

In [ ]:
pipe_ridgecv = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),
                             StandardScaler(),
                             Ridge(alpha=lambda_, random_state=1)) # pipe 3 degrees changing lambda
list_models.append((pipe_ridgecv,lambda_, 'Ridge'))

In [69]:
list_models = []
MSE = []
list_MSE = []

def modelval(dataframe):
    #Define feature matrix X and target vector y 
    
    X = pd.DataFrame(data=dataframe['data'], 
                     columns=dataframe['feature_names'])\
                 .iloc[:,:-2]
    y = dataframe['target']

    
    #Split X, y into development (2/3) and test data (1/3).
    #Then split the development data into train (1/2) and validation (1/2) data.
    X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=1)    
    X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=1/2, random_state=1)
    
    # Appends linear regression model
    pipe_linreg = make_pipeline(PolynomialFeatures(degree=1, include_bias=False),
                                                   StandardScaler(),
                                                   LinearRegression())
    list_models.append((pipe_linreg, 'linear', 'Linear'))
    
       # Hyperparameters
    lambdas =  np.logspace(-4, 4, 12)
    
    # Appends Lasso moddels with 
    for lambda_ in lambdas:
        pipe_lassocv = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),
                                     StandardScaler(),
                                     Lasso(alpha=lambda_, random_state=1)) # pipe 3 degrees changing lambda
        list_models.append((pipe_lassocv,lambda_, 'Lasso'))

    # Creating folds
    kfolds = KFold(n_splits=10, random_state=1)
    folds = list(kfolds.split(X_dev, y_dev))
    
    
    for k in list_models:
        pipe = k[0]
        j = 0
        for i, val, in folds: # looping over folds
            j += 1
            X_train, y_train = np.array(X_dev)[i], np.array(y_dev)[i] #np.array if X_dev is a pd.DataFrame
            X_val, y_val = np.array(X_dev)[val], np.array(y_dev)[val] #np.array if X_dev is a pd.DataFrame
            pipe.fit(X_train, y_train)        
            MSE.append((mse(pipe.predict(X_val), y_val), k[1], k[2], j))
    return MSE

In [70]:
testliste = modelval(cal_house)

C:\Users\magnus\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
C:\Users\magnus\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2588.2907980805917, tolerance: 1.6537003269567894
  positive)
C:\Users\magnus\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2556.1739833464408, tolerance: 1.650315875502991
  positive)
C:\Users\magnus\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number o

In [71]:
df_validation = pd.DataFrame(testliste, columns = ['MSE', 'Lambda', 'Modeltype', 'fold'])
df_validation = df_validation.set_index('Lambda')
df_validation_1 = pd.pivot_table(df_validation, values = 'MSE', index=['Lambda', 'Modeltype'], columns = 'fold').reset_index().set_index('Lambda')


optimal = df_validation.nsmallest(1,['MSE'])

In [72]:
df_validation_1

fold,Modeltype,1,2,3,4,5,6,7,8,9,10
Lambda,,,,,,,,,,,
0.0001,Lasso,0.491438,0.531207,555.720040,0.518805,0.503690,0.468580,6.098931,5.346069,0.559037,0.915056
0.0005336699231206312,Lasso,0.486274,0.536249,356.384726,0.520460,0.507165,0.473109,4.662336,1.319112,0.562481,2.911227
0.002848035868435802,Lasso,0.510849,0.560873,44.621451,0.546274,0.534419,0.498520,2.228514,0.688903,0.590105,1.319696
0.01519911082952933,Lasso,0.576658,0.601305,0.703595,0.612906,0.597113,0.555468,0.742085,0.682641,0.632839,0.798540
0.08111308307896872,Lasso,0.665366,0.685164,0.698005,0.684742,0.683036,0.617610,0.655072,0.632813,0.702557,0.644505
0.43287612810830617,Lasso,0.923165,0.925754,0.920287,0.856913,0.907661,0.824441,0.866634,0.833910,0.923725,0.863674
2.310129700083158,Lasso,1.408500,1.432963,1.368750,1.252642,1.368488,1.294850,1.344923,1.257251,1.351192,1.348036
12.32846739442066,Lasso,1.408500,1.432963,1.368750,1.252642,1.368488,1.294850,1.344923,1.257251,1.351192,1.348036
65.79332246575683,Lasso,1.408500,1.432963,1.368750,1.252642,1.368488,1.294850,1.344923,1.257251,1.351192,1.348036


In [41]:
print('Optimal lambda:', optimal.index[0])

Optimal lambda: 0.0001


In [47]:
X = pd.DataFrame(data=cal_house['data'], 
                 columns=cal_house['feature_names'])\
             .iloc[:,:-2]
y = cal_house['target']

X

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467
...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209
